In [33]:
from datasets import load_dataset, Audio
shanghai_corpus = load_dataset("TingChen-ppmc/Shanghai_Dialect_Conversational_Speech_Corpus")
mandarin_corpus = load_dataset("urarik/free_st_chinese_mandarin_corpus", split="train", streaming=True)
# note: this prevents a download which exceeds my computer space limit 




Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

## Process Dataset

In [34]:
import librosa
import librosa.display
import IPython.display as ipd
from pydub import AudioSegment
from pydub.utils import mediainfo
import scipy
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
from tqdm import tqdm

In [35]:
# Iterate over the mandarin_corpus dataset since its streaming
# Each row is a dictionary with keys: 'audio' and 'sentence'
for i, row in enumerate(tqdm(mandarin_corpus, desc="Processing Mandarin Corpus")):
    print(row)  # Each row is a dictionary with keys: 'audio' and 'sentence'
    
    # Access specific fields
    audio_data = row['audio']  # Contains audio information
    sentence = row['sentence']  # Contains the corresponding text transcription
    
    # Stop after a few rows; train dataset is 92k rows
    if i >= 10:  # Adjust the number as needed
        break

Processing Mandarin Corpus: 10it [00:02,  4.20it/s]

{'audio': {'path': '20170001P00231I0069.wav', 'array': array([ 4.88281250e-04,  4.88281250e-04,  2.44140625e-04, ...,
        0.00000000e+00, -6.10351562e-05, -4.57763672e-04]), 'sampling_rate': 16000}, 'sentence': '拥护不拥护照样是过日子'}
{'audio': {'path': '20170001P00413A0017.wav', 'array': array([ 4.88281250e-04,  4.88281250e-04,  2.44140625e-04, ...,
        0.00000000e+00, -6.10351562e-05, -4.57763672e-04]), 'sampling_rate': 16000}, 'sentence': '去买个行军床睡起来比较舒服'}
{'audio': {'path': '20170001P00103I0112.wav', 'array': array([ 4.88281250e-04,  4.88281250e-04,  2.44140625e-04, ...,
        0.00000000e+00, -6.10351562e-05, -4.57763672e-04]), 'sampling_rate': 16000}, 'sentence': '我在想你准备结婚的乖乖是谁呢'}
{'audio': {'path': '20170001P00062A0008.wav', 'array': array([ 4.88281250e-04,  4.88281250e-04,  2.44140625e-04, ...,
        0.00000000e+00, -6.10351562e-05, -4.57763672e-04]), 'sampling_rate': 16000}, 'sentence': '卡布达长大的七龙珠葫芦娃'}
{'audio': {'path': '20170001P00412A0009.wav', 'array': array([ 4.88281250e

In [36]:
shanghai_corpus

DatasetDict({
    train: Dataset({
        features: ['audio', 'gender', 'speaker_id', 'transcription'],
        num_rows: 3792
    })
})

In [ ]:
#pulls out audio, sample, transcription, and label from datasets and organizes them into a list of tuples
def process_shanghai(shanghai_corpus):
    data = []
    for row in shanghai_corpus['train']:
        audio = row['audio']['array']
        sampling_rate = row['audio']['sampling_rate']
        label = 0  # Label for Shanghai dataset
        text = row.get('transcription', '')  # Get transcription if available
        data.append((audio, sampling_rate, label, text))
    return data

def process_mandarin(mandarin_corpus, max_samples=3000):
    data = []
    for i, row in enumerate(mandarin_corpus):
        audio = row['audio']['array']
        sampling_rate = row['audio']['sampling_rate']
        label = 1  # Label for Mandarin dataset
        text = row.get('sentence', '')  # Get sentence if available
        data.append((audio, sampling_rate, label, text))
        if i >= max_samples - 1:
            break
    return data

# Process both datasets
shanghai_data = process_shanghai(shanghai_corpus)
mandarin_data = process_mandarin(mandarin_corpus)

# Combine the datasets
combined_data = shanghai_data + mandarin_data

# print the first 5 samples with text
for audio, sampling_rate, label, text in combined_data[-5:]: 
    print(f"Label: {label}, Text: {text[:30]}...")  # Print first 30 chars for brevity

SyntaxError: expected ':' (3826463841.py, line 32)

In [42]:
# Print the first 5 rows where the label is 1
count = 0
for audio, sampling_rate, label, text in combined_data:
    if label == 1:  # Check if the label is 2 (Mandarin dataset)
        print(f"Audio: {audio[:10]}...")  # Print the first 10 samples of the audio array for brevity
        print(f"Sampling Rate: {sampling_rate}")
        print(f"Label: {label}")
        print(f"Text: {text[:30]}...")  # Print the first 30 chars of the text for brevity
        print("-" * 50)
        count += 1
        if count >= 5:  # Stop after printing 5 rows
            break

Audio: [0.00048828 0.00048828 0.00024414 0.00012207 0.0005188  0.00079346
 0.00088501 0.00115967 0.0010376  0.00045776]...
Sampling Rate: 16000
Label: 1
Text: 拥护不拥护照样是过日子...
--------------------------------------------------
Audio: [0.00048828 0.00048828 0.00024414 0.00012207 0.0005188  0.00079346
 0.00088501 0.00115967 0.0010376  0.00045776]...
Sampling Rate: 16000
Label: 1
Text: 去买个行军床睡起来比较舒服...
--------------------------------------------------
Audio: [0.00048828 0.00048828 0.00024414 0.00012207 0.0005188  0.00079346
 0.00088501 0.00115967 0.0010376  0.00045776]...
Sampling Rate: 16000
Label: 1
Text: 我在想你准备结婚的乖乖是谁呢...
--------------------------------------------------
Audio: [0.00048828 0.00048828 0.00024414 0.00012207 0.0005188  0.00079346
 0.00088501 0.00115967 0.0010376  0.00045776]...
Sampling Rate: 16000
Label: 1
Text: 卡布达长大的七龙珠葫芦娃...
--------------------------------------------------
Audio: [0.00048828 0.00048828 0.00024414 0.00012207 0.0005188  0.00079346
 0.00088501 0.001159

## Detect gender feature on dataset via pre-trained model

In [43]:

# Adding gender detection using a pre-trained model

from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2FeatureExtractor


# Load model and processor
model_name = "prithivMLmods/Common-Voice-Gender-Detection"
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model.eval()

# Label mapping
id2label = {
    0: "female",
    1: "male"
}

def predict_gender(audio_array, sampling_rate):
    # Resample to 16kHz if needed
    if sampling_rate != 16000:
        audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=16000)
        sampling_rate = 16000
    # Prepare input
    inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
        pred_id = logits.argmax(dim=-1).item()
    return id2label[pred_id]

In [44]:
# Run gender prediction on the first 10 samples only
test_data = combined_data[:10]  # Get the first 10 samples

test_data_with_gender = []
for audio, sampling_rate, label, text in tqdm(test_data, desc="Gender Detection (Unit Test)"):
    gender = predict_gender(audio, sampling_rate)
    test_data_with_gender.append((audio, sampling_rate, label, text, gender))

# Print the results for inspection
for row in test_data_with_gender:
    print(f"Label: {row[2]}, Text: {row[3][:30]}, Gender: {row[4]}")

Gender Detection (Unit Test): 100%|██████████| 10/10 [00:02<00:00,  3.91it/s]

Label: 0, Text: 北京爱数智慧语音采集, Gender: male
Label: 0, Text: 北京爱数智慧语音采集, Gender: female
Label: 0, Text: 阿拉两个拧来聊聊金融方面呃, Gender: male
Label: 0, Text: 金融方面嘛, Gender: male
Label: 0, Text: 搿呃，阿姨喃，应该讲，侬已经交关年数辣辣了解了, Gender: male
Label: 0, Text: 葛末，吾辣辣金融方面已经有的三四年了, Gender: male
Label: 0, Text: 最少辰光阿拉是做撒呃喃，有钞票就是到银行里保本保息, Gender: male
Label: 0, Text: 吾已经做了已经到八七年了, Gender: female
Label: 0, Text: 八七年呃，当时辰光辣里哴相做理财呃, Gender: female
Label: 0, Text: 是两级风险，三级风险，四级风险吾侪做呃，信托咯撒侪做呃, Gender: female


In [25]:
# Example: Add gender to your combined_data
# combined_data = [(audio, sampling_rate, label, text), ...]
combined_data_with_gender = []
for audio, sampling_rate, label, text in tqdm(combined_data, desc="Gender Detection"):
    gender = predict_gender(audio, sampling_rate)
    combined_data_with_gender.append((audio, sampling_rate, label, text, gender))

# Now each tuple is (audio, sampling_rate, label, text, gender)
# Example: print the first 5 with gender
for row in combined_data_with_gender[:5]:
    print(f"Label: {row[2]}, Text: {row[3]}, Gender: {row[4]}")

Gender Detection: 100%|██████████| 6792/6792 [17:44<00:00,  6.38it/s]

Label: 0, Text: 北京爱数智慧语音采集, Gender: male
Label: 0, Text: 北京爱数智慧语音采集, Gender: female
Label: 0, Text: 阿拉两个拧来聊聊金融方面呃, Gender: male
Label: 0, Text: 金融方面嘛, Gender: male
Label: 0, Text: 搿呃，阿姨喃，应该讲，侬已经交关年数辣辣了解了, Gender: male


#### check accuracy of gender labelling on shanghai dataset

In [26]:


# Compare predicted gender to true gender label in Shanghai corpus
correct = 0
total = 0

for row in tqdm(shanghai_corpus['train'], desc="Evaluating Gender Detection"):
    audio = row['audio']['array']
    sampling_rate = row['audio']['sampling_rate']
    true_gender = row['gender'].lower()  # Ensure lowercase for comparison
    pred_gender = predict_gender(audio, sampling_rate)
    if pred_gender == true_gender:
        correct += 1
    total += 1

accuracy = correct / total if total > 0 else 0
print(f"Gender detection accuracy on Shanghai corpus: {accuracy * 100:.2f}%")

Evaluating Gender Detection: 100%|██████████| 3792/3792 [11:10<00:00,  5.65it/s]

Gender detection accuracy on Shanghai corpus: 86.76%


In [45]:
combined_data_with_gender

[(array([-0.00180054, -0.00204468, -0.00152588, ...,  0.00152588,
          0.00137329,  0.00067139]),
  16000,
  0,
  '北京爱数智慧语音采集',
  'male'),
 (array([0.00137329, 0.00094604, 0.        , ..., 0.00183105, 0.00213623,
         0.0017395 ]),
  16000,
  0,
  '北京爱数智慧语音采集',
  'female'),
 (array([ 0.00039673,  0.00054932,  0.00094604, ..., -0.00100708,
         -0.0005188 , -0.00018311]),
  16000,
  0,
  '阿拉两个拧来聊聊金融方面呃',
  'male'),
 (array([ 0.00125122,  0.00115967,  0.00131226, ..., -0.00317383,
         -0.00082397, -0.00186157]),
  16000,
  0,
  '金融方面嘛',
  'male'),
 (array([-0.00210571, -0.00137329, -0.00115967, ...,  0.0012207 ,
          0.00149536,  0.0015564 ]),
  16000,
  0,
  '搿呃，阿姨喃，应该讲，侬已经交关年数辣辣了解了',
  'male'),
 (array([ 3.05175781e-05,  4.27246094e-04,  0.00000000e+00, ...,
         -3.35693359e-04, -9.15527344e-05, -4.88281250e-04]),
  16000,
  0,
  '葛末，吾辣辣金融方面已经有的三四年了',
  'male'),
 (array([-0.0007019 , -0.00076294, -0.00061035, ...,  0.00146484,
          0.00183105,  0.002288

### other models to try: 

https://github.com/i0/voice-gender-detection

https://github.com/x4nth055/gender-recognition-by-voice


In [32]:
import pandas as pd

# combined_data_with_gender: [(audio, sampling_rate, label, text, gender), ...]
# For the classifier, you only need text and label (and optionally gender)

df = pd.DataFrame([
    {"sentence": text, "label": label, "gender": gender}
    for _, _, label, text, gender in combined_data_with_gender
])

# Save to TSV (tab-separated values)
df.to_csv("combined_data_with_gender.tsv", sep="\t", index=False)

# If you want to download it in Jupyter, use:
from IPython.display import FileLink
FileLink("combined_data_with_gender.tsv")

/Users/karenlu/Downloads/combined_data_with_gender.tsv

In [69]:
combined_data_with_gender

[(array([-0.00180054, -0.00204468, -0.00152588, ...,  0.00152588,
          0.00137329,  0.00067139]),
  16000,
  0,
  '北京爱数智慧语音采集',
  'male'),
 (array([0.00137329, 0.00094604, 0.        , ..., 0.00183105, 0.00213623,
         0.0017395 ]),
  16000,
  0,
  '北京爱数智慧语音采集',
  'female'),
 (array([ 0.00039673,  0.00054932,  0.00094604, ..., -0.00100708,
         -0.0005188 , -0.00018311]),
  16000,
  0,
  '阿拉两个拧来聊聊金融方面呃',
  'male'),
 (array([ 0.00125122,  0.00115967,  0.00131226, ..., -0.00317383,
         -0.00082397, -0.00186157]),
  16000,
  0,
  '金融方面嘛',
  'male'),
 (array([-0.00210571, -0.00137329, -0.00115967, ...,  0.0012207 ,
          0.00149536,  0.0015564 ]),
  16000,
  0,
  '搿呃，阿姨喃，应该讲，侬已经交关年数辣辣了解了',
  'male'),
 (array([ 3.05175781e-05,  4.27246094e-04,  0.00000000e+00, ...,
         -3.35693359e-04, -9.15527344e-05, -4.88281250e-04]),
  16000,
  0,
  '葛末，吾辣辣金融方面已经有的三四年了',
  'male'),
 (array([-0.0007019 , -0.00076294, -0.00061035, ...,  0.00146484,
          0.00183105,  0.002288

## Add sentiment

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "baidu/ERNIE-4.5-0.3B-Base-PT"
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)

# prompt = "Large language model is"
# model_inputs = tokenizer([prompt], add_special_tokens=False, return_tensors="pt").to(model.device)

# generated_ids = model.generate(
#     model_inputs.input_ids,
#     max_new_tokens=1024
# )
# result = tokenizer.decode(generated_ids[0].tolist(), skip_special_tokens=True)
# print("result:", result)


In [62]:
from transformers import BertTokenizer, ErnieModel

tokenizer = BertTokenizer.from_pretrained("nghuyong/ernie-3.0-nano-zh")
model = ErnieModel.from_pretrained("nghuyong/ernie-3.0-nano-zh")


Some weights of ErnieModel were not initialized from the model checkpoint at nghuyong/ernie-3.0-nano-zh and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
prompt = "你好我叫明光"
model_inputs = tokenizer(prompt)


In [67]:
model_inputs

{'input_ids': [1, 226, 170, 75, 951, 181, 325, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [68]:
tokenizer.decode(model_inputs.input_ids)

'[CLS] 你 好 我 叫 明 光 [SEP]'

In [70]:
#running across the data 
#data_v2 contains gender and token 

data_v2 = []
for audio, sampling_rate, label, text, gender in combined_data_with_gender: 
    token = tokenizer(text, padding='max_length', truncation=True, max_length = 128, return_tensors="pt")
    data_v2.append((audio, sampling_rate, label, text, gender, token))
    

In [73]:
tokenizer.decode(data_v2[0][5].input_ids[0], skip_special_tokens=True)

'北 京 爱 数 智 慧 语 音 采 集'

### additional tokenizer

In [53]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Sentiment analysis 


In [74]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch

tokenizer=BertTokenizer.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')
model=BertForSequenceClassification.from_pretrained('IDEA-CCNL/Erlangshen-Roberta-110M-Sentiment')

text='今天心情不好'

output=model(torch.tensor([tokenizer.encode(text)]))
print(torch.nn.functional.softmax(output.logits,dim=-1))


vocab.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

tensor([[0.9551, 0.0449]], grad_fn=<SoftmaxBackward0>)


In [79]:
def get_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128, padding="max_length")
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        sentiment = torch.argmax(probs, dim=-1).item()
    return sentiment  

data_v2_with_sentiment = []
for audio, sampling_rate, label, text, gender, token in data_v2[0:10]:  # Limit to first 10 for testing
    sentiment = get_sentiment(text)
    data_v2_with_sentiment.append((audio, sampling_rate, label, text, gender, token, sentiment))

# Example: print first 3 rows with sentiment
for row in data_v2_with_sentiment[:3]:
    print(f"Text: {row[3][:30]}, Gender: {row[4]}, Sentiment: {row[6]}")

Text: 北京爱数智慧语音采集, Gender: male, Sentiment: 1
Text: 北京爱数智慧语音采集, Gender: female, Sentiment: 1
Text: 阿拉两个拧来聊聊金融方面呃, Gender: male, Sentiment: 1


### large tokenizer

In [47]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "baidu/ERNIE-4.5-0.3B-Base-PT"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_ernie4_5.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baidu/ERNIE-4.5-0.3B-Base-PT:
- tokenization_ernie4_5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

A new version of the following files was downloaded from https://huggingface.co/baidu/ERNIE-4.5-0.3B-Base-PT:
- configuration_ernie4_5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_ernie4_5.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/baidu/ERNIE-4.5-0.3B-Base-PT:
- modeling_ernie4_5.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/722M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

## add generated transcription (mandarin)

In [29]:
import re
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from jiwer import wer, cer

MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn"
CHARS_TO_IGNORE = [",", "?", "¿", ".", "!", "¡", ";", "；", ":", '""', "%", '"', "�", "ʿ", "·", "჻", "~", "՞",
                  "؟", "،", "।", "॥", "«", "»", "„", "“", "”", "「", "」", "‘", "’", "《", "》", "(", ")", "[", "]",
                  "{", "}", "=", "`", "_", "+", "<", ">", "…", "–", "°", "´", "ʾ", "‹", "›", "©", "®", "—", "→", "。",
                  "、", "﹂", "﹁", "‧", "～", "﹏", "，", "｛", "｝", "（", "）", "［", "］", "【", "】", "‥", "〽",
                  "『", "』", "〝", "〟", "⟨", "⟩", "〜", "：", "！", "？", "♪", "；", "/", "\\", "º", "−", "^", "'", "ʻ", "ˆ"]
chars_to_ignore_regex = f"[{re.escape(''.join(CHARS_TO_IGNORE))}]"

# Load ASR model and processor
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
asr_model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
asr_model.eval()

def clean_text(text):
    # Remove special characters and convert to uppercase for fair comparison
    return re.sub(chars_to_ignore_regex, "", text).upper()

def transcribe(audio_array, sampling_rate):
    # Resample to 16kHz if needed
    if sampling_rate != 16000:
        audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=16000)
        sampling_rate = 16000
    # Normalize audio
    if audio_array.dtype != 'float32':
        audio_array = audio_array.astype('float32')
    # Prepare input
    input_values = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt").input_values
    with torch.no_grad():
        logits = asr_model(input_values).logits
        pred_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(pred_ids)[0]
    return transcription.strip()

# Build new dataset with reference, prediction, label, and gender
asr_results = []
for audio, sampling_rate, label, text, gender in tqdm(combined_data_with_gender[:10], desc="ASR Unit Test"):  # Use [:10] for unit test, remove for full dataset
    pred_text = transcribe(audio, sampling_rate)
    asr_results.append({
        "reference": clean_text(text),
        "prediction": clean_text(pred_text),
        "label": label,
        "gender": gender
    })

# Separate Shanghai and Mandarin results
shanghai_refs = [row["reference"] for row in asr_results if row["label"] == 0]
shanghai_preds = [row["prediction"] for row in asr_results if row["label"] == 0]
mandarin_refs = [row["reference"] for row in asr_results if row["label"] == 1]
mandarin_preds = [row["prediction"] for row in asr_results if row["label"] == 1]

# Compute metrics
shanghai_wer = wer(shanghai_refs, shanghai_preds) if shanghai_refs else None
shanghai_cer = cer(shanghai_refs, shanghai_preds) if shanghai_refs else None
mandarin_wer = wer(mandarin_refs, mandarin_preds) if mandarin_refs else None
mandarin_cer = cer(mandarin_refs, mandarin_preds) if mandarin_refs else None

print(f"Shanghai WER: {shanghai_wer:.3f}" if shanghai_wer is not None else "No Shanghai samples")
print(f"Shanghai CER: {shanghai_cer:.3f}" if shanghai_cer is not None else "No Shanghai samples")
print(f"Mandarin WER: {mandarin_wer:.3f}" if mandarin_wer is not None else "No Mandarin samples")
print(f"Mandarin CER: {mandarin_cer:.3f}" if mandarin_cer is not None else "No Mandarin samples")

# Optional: inspect results
for row in asr_results:
    print(f"Label: {row['label']}, Gender: {row['gender']}")
    print(f"Reference: {row['reference']}")
    print(f"Prediction: {row['prediction']}")
    print("-" * 40)

ASR Unit Test: 100%|██████████| 10/10 [00:07<00:00,  1.26it/s]

Shanghai WER: 1.000
Shanghai CER: 0.894
No Mandarin samples
No Mandarin samples
Label: 0, Gender: male
Reference: 北京爱数智慧语音采集
Prediction: 不近也书资会许域切间
----------------------------------------
Label: 0, Gender: female
Reference: 北京爱数智慧语音采集
Prediction: 破听UNK手智慧理且救
----------------------------------------
Label: 0, Gender: male
Reference: 阿拉两个拧来聊聊金融方面呃
Prediction: 安了脸岸历列有着金用方闭的
----------------------------------------
Label: 0, Gender: male
Reference: 金融方面嘛
Prediction: 真入发给外来
----------------------------------------
Label: 0, Gender: male
Reference: 搿呃阿姨喃应该讲侬已经交关年数辣辣了解了
Prediction: 和爱意的应给段奴京这被理素了了表加来
----------------------------------------
Label: 0, Gender: male
Reference: 葛末吾辣辣金融方面已经有的三四年了
Prediction: 跟么无来的近方比经的谢死年了
----------------------------------------
Label: 0, Gender: male
Reference: 最少辰光阿拉是做撒呃喃有钞票就是到银行里保本保息
Prediction: 据说有猛光阿来主杀呢有扯破了子斗你那里薄泵抱士
----------------------------------------
Label: 0, Gender: female
Reference: 吾已经做了已经到八七年了
Prediction: 我经足累经的霸劝一来
-----------------------------

In [30]:
combined_data_with_gender

[(array([-0.00180054, -0.00204468, -0.00152588, ...,  0.00152588,
          0.00137329,  0.00067139]),
  16000,
  0,
  '北京爱数智慧语音采集',
  'male'),
 (array([0.00137329, 0.00094604, 0.        , ..., 0.00183105, 0.00213623,
         0.0017395 ]),
  16000,
  0,
  '北京爱数智慧语音采集',
  'female'),
 (array([ 0.00039673,  0.00054932,  0.00094604, ..., -0.00100708,
         -0.0005188 , -0.00018311]),
  16000,
  0,
  '阿拉两个拧来聊聊金融方面呃',
  'male'),
 (array([ 0.00125122,  0.00115967,  0.00131226, ..., -0.00317383,
         -0.00082397, -0.00186157]),
  16000,
  0,
  '金融方面嘛',
  'male'),
 (array([-0.00210571, -0.00137329, -0.00115967, ...,  0.0012207 ,
          0.00149536,  0.0015564 ]),
  16000,
  0,
  '搿呃，阿姨喃，应该讲，侬已经交关年数辣辣了解了',
  'male'),
 (array([ 3.05175781e-05,  4.27246094e-04,  0.00000000e+00, ...,
         -3.35693359e-04, -9.15527344e-05, -4.88281250e-04]),
  16000,
  0,
  '葛末，吾辣辣金融方面已经有的三四年了',
  'male'),
 (array([-0.0007019 , -0.00076294, -0.00061035, ...,  0.00146484,
          0.00183105,  0.002288

## Run pre-trained classification model

In [101]:
# Function to extract features
def extract_features(audio, sr, n_mfcc=40):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return np.transpose(mfccs)  # Transpose to match (time, features) format

# Preprocess the combined dataset with tqdm
processed_data = []
for audio, sr, label in tqdm(combined_data, desc="Processing Combined Dataset"):
    features = extract_features(audio, sr)
    processed_data.append((features, label))

Processing Combined Dataset: 100%|██████████| 6792/6792 [00:35<00:00, 190.25it/s]


In [102]:
processed_data[:5]

[(array([[-5.75503837e+02,  8.88032345e+01,  4.50311762e+01, ...,
          -6.20060546e-01,  2.71644276e+00,  2.84673396e+00],
         [-5.51143555e+02,  8.79621858e+01,  4.19003920e+01, ...,
           1.24691137e+00,  2.39314673e+00,  8.49372334e-01],
         [-5.51453888e+02,  8.62375973e+01,  4.32768060e+01, ...,
          -3.93854645e-01,  8.76537031e-01,  2.08334142e+00],
         ...,
         [-5.33971384e+02,  8.96917373e+01,  4.43571294e+01, ...,
          -6.50467565e+00, -8.31660028e+00, -4.41273894e+00],
         [-5.06438269e+02,  8.59629943e+01,  2.45915072e+01, ...,
          -6.65721862e+00, -2.99549514e+00, -4.50415327e+00],
         [-4.79388851e+02,  7.17054078e+01,  8.47340807e-01, ...,
          -7.71282908e+00, -5.43654767e-01, -4.41271713e+00]]),
  0),
 (array([[-5.35011309e+02,  6.33103671e+01,  3.91639815e+01, ...,
          -1.68727067e+00, -5.11195089e+00,  4.66895520e+00],
         [-5.01899057e+02,  6.78334631e+01,  4.21930026e+01, ...,
           9.439

In [103]:
# Check the labels in processed_data
labels = [label for _, label in processed_data]
print(set(labels))  # Should print {0, 1}

{0, 1}


In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]=""
device = torch.device("cpu")
#force CPU usage; no GPU available


In [ ]:
loaded_model = torch.load('model9.model', map_location=device)
#download from https://github.com/Colt1990/chinese-dialect-recognition/blob/master/model9.model

In [29]:
print(loaded_model.keys())

odict_keys(['layer1.GRU.weight_ih_l0', 'layer1.GRU.weight_hh_l0', 'layer1.GRU.bias_ih_l0', 'layer1.GRU.bias_hh_l0', 'layer1.GRU.weight_ih_l1', 'layer1.GRU.weight_hh_l1', 'layer1.GRU.bias_ih_l1', 'layer1.GRU.bias_hh_l1', 'layer2.linear.weight', 'layer2.linear.bias', 'layer3.linear.weight', 'layer3.linear.bias'])


In [30]:
print(loaded_model)

OrderedDict([('layer1.GRU.weight_ih_l0', tensor([[ 0.0807, -0.0464, -0.0200,  ..., -0.0025, -0.0084, -0.1005],
        [ 0.0160, -0.1390, -0.1184,  ...,  0.0214,  0.1020,  0.1212],
        [-0.0401, -0.0430, -0.0914,  ..., -0.0020, -0.0807,  0.0251],
        ...,
        [-0.0387, -0.5996, -0.2949,  ..., -0.0725, -0.0012, -0.3281],
        [-0.5621, -0.2069, -0.2013,  ..., -0.0044,  0.0285,  0.0524],
        [-0.5540,  0.3632,  0.1798,  ..., -0.0338, -0.0958, -0.3366]])), ('layer1.GRU.weight_hh_l0', tensor([[ 6.7413e-03, -1.8119e-02,  9.9958e-03,  ..., -5.2676e-02,
          3.2840e-05, -2.8702e-02],
        [ 3.0212e-02,  8.7385e-03, -1.6029e-02,  ..., -2.0702e-02,
          8.8958e-02, -4.0659e-02],
        [-6.2419e-02, -8.2796e-02, -4.7482e-02,  ...,  5.6611e-02,
         -5.9157e-02, -4.4292e-02],
        ...,
        [-1.2909e-02, -3.1017e-02, -1.2714e-02,  ...,  9.0505e-03,
         -7.6070e-02,  3.5961e-02],
        [ 5.9787e-02,  1.1647e-01,  5.3197e-02,  ..., -1.4718e-02,
   

In [104]:
from torch.utils.data import DataLoader, TensorDataset

# Prepare tensors
features = [torch.tensor(f, dtype=torch.float32) for f, _ in processed_data]
labels = [torch.tensor(l, dtype=torch.long) for _, l in processed_data]

# Pad sequences to the same length
from torch.nn.utils.rnn import pad_sequence
padded_features = pad_sequence(features, batch_first=True)
labels = torch.tensor(labels)

# Create a DataLoader
dataset = TensorDataset(padded_features, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [106]:
## LanNet Model: 

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class LanNet(nn.Module):
    def __init__(self, input_dim=40, hidden_dim=512, bn_dim=192, output_dim=2):
        super(LanNet, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.bn_dim = bn_dim
        self.output_dim = output_dim

        self.layer1 = nn.Sequential()
        self.layer1.add_module('GRU', nn.GRU(self.input_dim, self.hidden_dim, num_layers=2, batch_first=True, bidirectional=False))
	
	
        self.layer2 = nn.Sequential()
        self.layer2.add_module('linear', nn.Linear(self.hidden_dim, self.bn_dim))

        self.layer3 = nn.Sequential()
        self.layer3.add_module('linear', nn.Linear(self.bn_dim,self.output_dim))

    # def forward(self, src, mask, target):
    #     batch_size, fea_frames, fea_dim = src.size()

    #     out_hidden, hidd = self.layer1(src)
    #     out_hidden = out_hidden.contiguous().view(-1, out_hidden.size(-1))   
    #     out_bn = self.layer2(out_hidden)
    #     out_target = self.layer3(out_bn)


    #     out_target = out_target.contiguous().view(batch_size, fea_frames, -1)
    #     mask = mask.contiguous().view(batch_size, fea_frames, 1).expand(batch_size, fea_frames, out_target.size(2))
    #     out_target_mask = out_target * mask
    #     out_target_mask = out_target_mask.sum(dim=1)/mask.sum(dim=1)
    #     predict_target = F.softmax(out_target_mask, dim=1)

    #     # Reshape target to match the dimensions of predict_target
    #     target = target.view(-1, 1)  # Reshape to [batch_size, 1]
	

    #     # 计算loss
    #     tar_select_new = torch.gather(predict_target, 1, target)
    #     ce_loss = -torch.log(tar_select_new) 
    #     ce_loss = ce_loss.sum() / batch_size

    #     # 计算acc
    #     _, predict = predict_target.max(dim=1)
    #     predict = predict.contiguous().view(-1,1)
    #     correct = predict.eq(target).float()       
    #     num_samples = predict.size(0)
    #     sum_acc = correct.sum().item()
    #     acc = sum_acc/num_samples

    #     return acc, ce_loss
    
    def forward(self, src, mask, target=None):
        batch_size, fea_frames, fea_dim = src.size()

        out_hidden, hidd = self.layer1(src)
        out_hidden = out_hidden.contiguous().view(-1, out_hidden.size(-1))   
        out_bn = self.layer2(out_hidden)
        out_target = self.layer3(out_bn)

        out_target = out_target.contiguous().view(batch_size, fea_frames, -1)
        mask = mask.contiguous().view(batch_size, fea_frames, 1).expand(batch_size, fea_frames, out_target.size(2))
        out_target_mask = out_target * mask
        out_target_mask = out_target_mask.sum(dim=1) / mask.sum(dim=1)
        predict_target = F.softmax(out_target_mask, dim=1)

        if target is None:
            # During evaluation, return only the predicted probabilities
            return predict_target

        # Reshape target to match the dimensions of predict_target
        target = target.view(-1, 1)  # Reshape to [batch_size, 1]

        # Compute loss
        tar_select_new = torch.gather(predict_target, 1, target)
        ce_loss = -torch.log(tar_select_new) 
        ce_loss = ce_loss.sum() / batch_size

        # Compute accuracy
        _, predict = predict_target.max(dim=1)
        predict = predict.contiguous().view(-1, 1)
        correct = predict.eq(target).float()
        num_samples = predict.size(0)
        sum_acc = correct.sum().item()
        acc = sum_acc / num_samples

        return acc, ce_loss

    

In [107]:
# Define the LanNet model
model = LanNet(input_dim=40, hidden_dim=512, bn_dim=192, output_dim=2)  # 2 classes: Shanghai and Mandarin
state_dict = torch.load('model9.model', map_location='cpu')

# Remove the weights for the final layer (layer3.linear) from the state_dict since we have only 2 classes
state_dict.pop('layer3.linear.weight')
state_dict.pop('layer3.linear.bias')

model.load_state_dict(state_dict, strict=False)  # Use strict=False to ignore missing keys
model.eval()  # Set the model to evaluation mode

LanNet(
  (layer1): Sequential(
    (GRU): GRU(40, 512, num_layers=2, batch_first=True)
  )
  (layer2): Sequential(
    (linear): Linear(in_features=512, out_features=192, bias=True)
  )
  (layer3): Sequential(
    (linear): Linear(in_features=192, out_features=2, bias=True)
  )
)

In [108]:

# Perform classification with tqdm
with torch.no_grad():
    for batch_features, batch_labels in tqdm(dataloader, desc="Classifying Batches"):
        mask = torch.ones(batch_features.size(0), batch_features.size(1), dtype=torch.float32)  # Create masks
        predictions, _ = model(batch_features, mask, batch_labels)

Classifying Batches: 100%|██████████| 425/425 [01:18<00:00,  5.44it/s]


In [109]:
predictions

0.375

In [110]:
# from tqdm import tqdm

# # Initialize variables to track accuracy
# correct_predictions = 0
# total_samples = 0

# # Perform classification and calculate accuracy
# with torch.no_grad():
#     for batch_features, batch_labels in tqdm(dataloader, desc="Evaluating Accuracy"):
#         # Create masks (all ones, assuming no padding)
#         mask = torch.ones(batch_features.size(0), batch_features.size(1), dtype=torch.float32)

#         # Get predictions from the model
#         predictions, _ = model(batch_features, mask, batch_labels)

#         # Get the predicted classes
#         predicted_classes = torch.argmax(predictions, dim=1)  # Shape: [batch_size]

#         # Compare with ground truth labels
#         correct_predictions += (predicted_classes == batch_labels).sum().item()
#         total_samples += batch_labels.size(0)

# # Calculate accuracy
# accuracy = correct_predictions / total_samples
# print(f"Accuracy: {accuracy * 100:.2f}%")

from tqdm import tqdm

# Initialize variables to track accuracy
correct_predictions = 0
total_samples = 0

# Perform classification and calculate accuracy
with torch.no_grad():
    for batch_features, batch_labels in tqdm(dataloader, desc="Evaluating Accuracy"):
        # Ensure batch_labels has the correct shape and type
        batch_labels = batch_labels.view(-1)  # Flatten to [batch_size]
        batch_labels = batch_labels.long()  # Ensure type is torch.long

        # Create masks (all ones, assuming no padding)
        mask = torch.ones(batch_features.size(0), batch_features.size(1), dtype=torch.float32)

        # Get predictions from the model
        predictions = model(batch_features, mask)  # Only predicted probabilities are returned

        # Get the predicted classes
        predicted_classes = torch.argmax(predictions, dim=1)  # Shape: [batch_size]

        # Compare with ground truth labels
        correct_predictions += (predicted_classes == batch_labels).sum().item()
        total_samples += batch_labels.size(0)

# Calculate accuracy
accuracy = correct_predictions / total_samples
print(f"Accuracy: {accuracy * 100:.2f}%")

Evaluating Accuracy: 100%|██████████| 425/425 [01:15<00:00,  5.62it/s]

Accuracy: 54.51%


### Fine-tuning

Note that the dialect data sizes are similar. Thus, our accuracy is no better than a coin flip. Unfortunate. 

1. Need to sort out features and break out gender, length of audio, age to see if impacts predictive accuracy/classification
2. Fine-tune and adjust parameters to find improvements
3. Implement denoising model on all to see if improves 
4. Add additional classes (dialects) to see if classification improves 

In [11]:
# # test fine-tuning

# # Fine-tune the model
# model.train()
# for epoch in range(10):  # Number of epochs
#     for batch_features, batch_labels in dataloader:
#         # Ensure batch_labels has the correct shape and type
#         batch_labels = batch_labels.view(-1, 1)  # Reshape to [batch_size, 1]
#         batch_labels = batch_labels.long()  # Ensure type is torch.long

#         # Create a mask (all ones, as no padding is applied here)
#         mask = torch.ones(batch_features.size(0), batch_features.size(1), dtype=torch.float32)

#         # Forward pass
#         _, loss = model(batch_features, mask, batch_labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch + 1}, Loss: {loss.item()}")


## Analyze linguistic components 